In [ ]:
import sys
sys.path.append('../htm_rl/htm_rl/')

from agent.agent import Agent, AgentRunner
from agent.memory import Memory, TemporalMemory
from agent.planner import Planner
from envs.mdp import Mdp
from common.sa_sdr_encoder import SaSdrEncoder, format_sa_superposition
from common.int_sdr_encoder import IntSdrEncoder, IntSdrFormatter
from common.int_sdr_encoder import SequenceSdrEncoder, IntSemanticSdrEncoder
from envs.mymdp import GridWorld


import numpy as np
import matplotlib.pyplot as plt

Basic example for standard MDP environment

In [ ]:
mdp = Mdp(transitions = {
            0: {0: 4, 1: 1},
            1: {0: 1, 1: 2},
            2: {0: 2, 1: 3},
            3: {0: 3, 1: 0},
            4: None
        })

In [ ]:
mdp.n_states

In [ ]:
state_encoder = IntSdrEncoder('state',
                              n_values=mdp.n_states,
                              value_bits=10,
                              activation_threshold=7)


action_encoder = IntSdrEncoder('action',
                               n_values=mdp.n_actions,
                               value_bits=10,
                               activation_threshold=7)



In [ ]:
sa_encoder = SaSdrEncoder(state_encoder, action_encoder)

In [ ]:
sa_encoder.total_bits
sa_encoder.activation_threshold

In [ ]:
tm = TemporalMemory(n_columns=sa_encoder.total_bits,
                    cells_per_column=1,
                    activation_threshold=sa_encoder.activation_threshold,
                    learning_threshold=action_encoder.activation_threshold,
                    initial_permanence=0.49,
                    connected_permanence=0.5,
                    maxNewSynapseCount=sa_encoder.value_bits,
                    maxSynapsesPerSegment=sa_encoder.value_bits)

In [ ]:
sa_formatter = IntSdrFormatter()

In [ ]:
memory = Memory(tm, sa_encoder, sa_formatter, sa_encoder.format)

In [ ]:
planner = Planner(memory,
                  planning_horizon=10,
                  goal_memory_size=2)

In [ ]:
agent = Agent(memory, planner, mdp.n_actions)

In [ ]:
run = AgentRunner(agent, mdp,
                  n_episodes=100,
                  max_steps=10,
                  pretrain=50,
                  verbosity=1)

In [ ]:
run.run()

Example of Agent in gridworld environment with two-value state.


In [ ]:
world_description = [[2,0,0,0,0],
                     [1,1,1,0,0],
                     [0,0,0,0,1],
                     [1,1,0,0,0],
                     [0,0,0,0,0]]

In [ ]:
gw = GridWorld(world_description, (5, 5), agent_initial_position={'row': 4, 'column': 0})

In [ ]:
gw.world_size

In [ ]:
state_encoder = SequenceSdrEncoder('state', encoders=[IntSdrEncoder('distance', gw.world_size[0]+1, 10, 7),
                                                   IntSdrEncoder('surface', 3, 10, 7)],
                                size=2)

In [ ]:
print(state_encoder.value_bits)
state_encoder.total_bits

In [ ]:
action_encoder = IntSdrEncoder('action', 3,
                              value_bits=10, activation_threshold=7)

In [ ]:
sa_encoder = SaSdrEncoder(state_encoder, action_encoder)

In [ ]:
tm = TemporalMemory(n_columns=sa_encoder.total_bits,
                    cells_per_column=2,
                    activation_threshold=sa_encoder.activation_threshold,
                    learning_threshold=action_encoder.activation_threshold,
                    initial_permanence=0.49,
                    connected_permanence=0.5)

In [ ]:
sa_formatter = IntSdrFormatter()

In [ ]:
memory = Memory(tm, sa_encoder, sa_formatter, format_sa_superposition)

In [ ]:
planner = Planner(memory, 10, 2)

In [ ]:
agent = Agent(memory, planner, gw.n_actions)

In [ ]:
run = AgentRunner(agent, gw, 5, 10, 0, 1)

In [ ]:
run.run()

Example of agent in two-value-state environment with semantic encoder


In [ ]:
world_description = [[2,0,0],
                     [1,1,0],
                     [0,0,0]]

In [ ]:
gw = GridWorld(world_description, (3, 3), agent_initial_position={'row': 2, 'column': 0})

In [ ]:
gw.world_size

In [ ]:
gw.render()

In [ ]:
state_encoder = SequenceSdrEncoder('state',
                                   encoders=[IntSdrEncoder('distance',
                                                                   gw.world_size[0],
                                                                   3,
                                                                   2),
                                                   IntSdrEncoder('surface', 3, 3, 2)],
                                   size=2)

In [ ]:
print(state_encoder.value_bits)
state_encoder.total_bits

In [ ]:
action_encoder = IntSdrEncoder('action', gw.n_actions,
                              value_bits=6, activation_threshold=4)

In [ ]:
sa_encoder = SaSdrEncoder(state_encoder, action_encoder)

In [ ]:
sa_encoder.total_bits, sa_encoder.activation_threshold

In [ ]:
action_encoder.activation_threshold

In [ ]:
tm = TemporalMemory(n_columns=sa_encoder.total_bits,
                    cells_per_column=8,
                    activation_threshold=sa_encoder.activation_threshold,
                    learning_threshold=action_encoder.activation_threshold,
                    initial_permanence=0.45,
                    connected_permanence=0.5,
                    maxNewSynapseCount=sa_encoder.value_bits,
                    maxSynapsesPerSegment=sa_encoder.value_bits)

In [ ]:
tm.activation_threshold, tm.connected_permanence

In [ ]:
tm.getMaxSegmentsPerCell(), tm.getMaxNewSynapseCount(), tm.getMaxSynapsesPerSegment()

In [ ]:
sdr_formatter = IntSdrFormatter()

In [ ]:
memory = Memory(tm, sa_encoder, sa_encoder.format, format_sa_superposition)

In [ ]:
planner = Planner(memory, 10, 1)

In [ ]:
planner.planning_horizon

In [ ]:
agent = Agent(memory, planner, gw.n_actions)

In [ ]:
agent._planning_horizon, agent._planning_enabled

In [ ]:
run = AgentRunner(agent, gw, 10000, 12, 10000, 1)

In [ ]:
run.run()


In [ ]:
run.agent.planner.inter_episode_goal_memory._set

In [ ]:
run.agent.set_planning_horizon(10)
run.max_steps = 12
run.verbosity = 1
run.pretrain = 0
run.n_episodes = 10
run.run()

In [ ]:
run.agent.set_planning_horizon(10)
run.max_steps = 10
run.verbosity = 3
run.pretrain = 0
run.n_episodes = 1
run.run()


In [ ]:
fig = plt.figure(figsize=(10, 7))
steps = np.array(run.train_stats.steps[10000:])
plt.plot(np.arange(steps.size), steps, '.')

In [ ]:
steps.mean(), steps.std()

In [ ]:
fig = plt.figure(figsize=(10, 7))
steps = np.array(run.train_stats.rewards[10000:])
plt.plot(np.arange(steps.size), steps, '.')
